In [1]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.17


In [12]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [13]:
from azureml.core import Workspace

ws = Workspace(subscription_id = '',
               resource_group = '',
               workspace_name = '')

ws.write_config()
print("Workspace configuration succeeded.")

Wrote the config file config.json to: /home/nbuser/library/aml_config/config.json
Workspace configuration succeeded.


In [14]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

Found existing compute target.
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-03-13T11:28:38.988000+00:00', 'creationTime': '2019-03-13T11:02:28.154998+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-03-13T11:02:49.322816+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [15]:
import os

project_folder = './pytorch-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

In [16]:
import shutil
shutil.copy('pytorch_horovod_mnist.py', project_folder)

'./pytorch-distr-hvd/pytorch_horovod_mnist.py'

In [17]:
from azureml.core import Experiment

experiment_name = 'pytorch-distr-hvd'
experiment = Experiment(ws, name=experiment_name)


In [18]:
from azureml.train.dnn import PyTorch

estimator = PyTorch(source_directory=project_folder,
                    compute_target=compute_target,
                    entry_script='pytorch_horovod_mnist.py',
                    node_count=2,
                    process_count_per_node=1,
                    distributed_backend='mpi',
                    use_gpu=True)


In [19]:
run = experiment.submit(estimator)
print(run)


Run(Experiment: pytorch-distr-hvd,
Id: pytorch-distr-hvd_1552476904_d42b0f15,
Type: azureml.scriptrun,
Status: Queued)


In [20]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

In [21]:
run.wait_for_completion(show_output=True) 

RunId: pytorch-distr-hvd_1552476904_d42b0f15

Streaming azureml-logs/60_control_log_rank_0.txt

This is an MPI job. Rank:0
Streaming log file azureml-logs/60_control_log_rank_0.txt

Streaming azureml-logs/80_driver_log_rank_0.txt



The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.1012263298034668 seconds

Execution Summary
RunId: pytorch-distr-hvd_1552476904_d42b0f15

Warnings:
Failed to upload Batch AI logs to RunHistory. The logs can be accessed directly on the file share .




{'runId': 'pytorch-distr-hvd_1552476904_d42b0f15',
 'target': 'gpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-03-13T11:39:23.489719Z',
 'endTimeUtc': '2019-03-13T11:41:29.131156Z',
 'warnings': [{'message': 'Failed to upload Batch AI logs to RunHistory. The logs can be accessed directly on the file share .'}],
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'fb97cbd3-5ae8-4e37-b71d-196dd6d146a9'},
 'runDefinition': {'Script': 'pytorch_horovod_mnist.py',
  'Arguments': [],
  'SourceDirectoryDataStore': 'workspaceblobstore',
  'Framework': 0,
  'Communicator': 5,
  'Target': 'gpucluster',
  'DataReferences': {'workspaceblobstore': {'DataStoreName': 'workspaceblobstore',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 2,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDep